
<p style="font-size:21px; font-family:'Times New Roman';">
Read report dates data 
</p>

In [1]:
import pandas as pd
import numpy as np
csv_file_path = '/Users/balmeru/Downloads/QQQQ.csv'
df = pd.read_csv(csv_file_path)


In [2]:
df['fyr'] = pd.to_numeric(df['fyr'], errors='coerce')
df = df.sort_values(by=['tic', 'fyearq', 'fqtr'])
ticker_change = df['tic'] != df['tic'].shift(1)
df['fyr_change_dummy'] = ((df['fyr'] != df['fyr'].shift(1)) & (~ticker_change)).astype(int)

# Fill the first occurrence of each ticker group with 0
df.loc[df.groupby('tic').head(1).index, 'fyr_change_dummy'] = 0


In [3]:
count_fyr_change_dummy_1 = df['fyr_change_dummy'].sum()
print(f"Number of times fyr_change_dummy is equal to 1: {count_fyr_change_dummy_1}")

Number of times fyr_change_dummy is equal to 1: 2188


In [4]:
nan_count = df[df['fyearq'].isna() | df['fqtr'].isna()].shape[0]

print("Number of rows where either fyearq or fqtr is NaN:", nan_count)


Number of rows where either fyearq or fqtr is NaN: 0


In [5]:
df_length = len(df)

print("Overall length of the DataFrame:", df_length)

Overall length of the DataFrame: 1085915


In [6]:
duplicated_cases = df[df.duplicated(subset=['tic', 'fyearq', 'fqtr'], keep=False)].shape[0]

print("Number of duplicated cases based on tic, fyearq, and fqtr:", duplicated_cases)


Number of duplicated cases based on tic, fyearq, and fqtr: 20779


In [7]:
duplicated_rows = df[df.duplicated(subset=['tic', 'fyearq', 'fqtr'], keep=False)]

# Count unique combinations of tic, fyearq, and fqtr among the duplicated rows
unique_duplicates_count = duplicated_rows.drop_duplicates(subset=['tic', 'fyearq', 'fqtr']).shape[0]

print("Number of unique duplicated combinations:", unique_duplicates_count)


Number of unique duplicated combinations: 10304


In [8]:
duplicated_rows = df[df.duplicated(subset=['tic', 'fyearq', 'fqtr'], keep=False)]

print("Rows where the same year and quarter is duplicated for the same ticker:")
print(duplicated_rows.head())


Rows where the same year and quarter is duplicated for the same ticker:
        GVKEY    datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
630256  20783  1992-03-31    1992     3    6   INDL      C      D     STD   
630257  20783  1992-03-31    1992     3    6   INDL      C      D     STD   
630258  20783  1992-06-30    1992     4    6   INDL      C      D     STD   
630259  20783  1992-06-30    1992     4    6   INDL      C      D     STD   
630260  20783  1992-09-30    1993     1    6   INDL      C      D     STD   

          tic  ...   dvpq     ibq      lseq       ltq   pstkq     teqq  txdbq  \
630256  0090A  ...  0.327  90.634  1938.769  1038.029     NaN  900.740    NaN   
630257  0090A  ...  0.327  90.634  1938.769  1038.029     NaN  900.740    NaN   
630258  0090A  ...  0.328  26.200  1932.309   997.431  28.846  934.878    NaN   
630259  0090A  ...  0.328  26.200  1932.309   997.431  28.846  934.878    NaN   
630260  0090A  ...  0.327  67.609  2046.837  1081.666     Na

In [9]:
df = df.drop_duplicates(subset=['tic', 'fyearq', 'fqtr'], keep='first')

print("DataFrame with only the first instances:")
print(df)


DataFrame with only the first instances:
        GVKEY    datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
60380    2484  1983-07-31    1983     3   10   INDL      C      D     STD   
60381    2484  1983-10-31    1983     4   10   INDL      C      D     STD   
60382    2484  1984-01-31    1984     1   10   INDL      C      D     STD   
60383    2484  1984-04-30    1984     2   10   INDL      C      D     STD   
60384    2484  1984-07-31    1984     3   10   INDL      C      D     STD   
...       ...         ...     ...   ...  ...    ...    ...    ...     ...   
782060  30165  1999-06-30    1999     2   12   INDL      C      D     STD   
782061  30165  1999-09-30    1999     3   12   INDL      C      D     STD   
782062  30165  1999-12-31    1999     4   12   INDL      C      D     STD   
782063  30165  2000-03-31    2000     1   12   INDL      C      D     STD   
782064  30165  2000-06-30    2000     2   12   INDL      C      D     STD   

          tic  ...  dvpq     ibq  

In [10]:
print(len(df))

1075440


In [11]:
1075440+20779-10304

1085915

In [12]:
initial_count = df['fyr_change_dummy'].sum()
print("Initial count of fyr_change_dummy = 1:", initial_count)            


Initial count of fyr_change_dummy = 1: 2139


In [13]:
def propagate_fyr_change_dummy(df):
    df_sorted = df.sort_values(by=['tic', 'fyearq', 'fqtr']).reset_index(drop=True)
    updated_indices = set()  # To track indices that are updated
    
    for index, row in df_sorted.iterrows():
        if row['fyr_change_dummy'] == 1:
            tic = row['tic']
            fyearq = row['fyearq']
            fqtr = row['fqtr']
            
            # Propagate to next 3 quarters
            for i in range(1, 4):
                next_fyearq = fyearq + (fqtr + i - 1) // 4  # Calculate next fiscal year
                next_fqtr = (fqtr + i - 1) % 4 + 1  # Calculate next quarter within the fiscal year
                
                # Find the index of the next quarter
                next_row_idx = df_sorted.index[(df_sorted['tic'] == tic) & (df_sorted['fyearq'] == next_fyearq) & (df_sorted['fqtr'] == next_fqtr)]
                
                if len(next_row_idx) > 0 and next_row_idx[0] not in updated_indices:
                    # Update fyr_change_dummy for the next quarter
                    df_sorted.at[next_row_idx[0], 'fyr_change_dummy'] = 1
                    updated_indices.add(next_row_idx[0])
    
    return df_sorted

# Apply the propagation functon
df_propagated = propagate_fyr_change_dummy(df)
df = df_propagated

# Print the propagated DataFrame
print("Propagated DataFrame:")
print(df_propagated)

Propagated DataFrame:
         GVKEY    datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
0         2484  1983-07-31    1983     3   10   INDL      C      D     STD   
1         2484  1983-10-31    1983     4   10   INDL      C      D     STD   
2         2484  1984-01-31    1984     1   10   INDL      C      D     STD   
3         2484  1984-04-30    1984     2   10   INDL      C      D     STD   
4         2484  1984-07-31    1984     3   10   INDL      C      D     STD   
...        ...         ...     ...   ...  ...    ...    ...    ...     ...   
1075435  30165  1999-06-30    1999     2   12   INDL      C      D     STD   
1075436  30165  1999-09-30    1999     3   12   INDL      C      D     STD   
1075437  30165  1999-12-31    1999     4   12   INDL      C      D     STD   
1075438  30165  2000-03-31    2000     1   12   INDL      C      D     STD   
1075439  30165  2000-06-30    2000     2   12   INDL      C      D     STD   

           tic  ...  dvpq     ibq      ls

In [14]:
final_count = df_propagated['fyr_change_dummy'].sum()

# Print the results
print("Expected max final count after propagation:", initial_count * 4)
print("Actual final count after propagation:", final_count)

Expected max final count after propagation: 8556
Actual final count after propagation: 8171


In [60]:
print(df)

         GVKEY    datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
0         2484  1983-07-31    1983     3   10   INDL      C      D     STD   
1         2484  1983-10-31    1983     4   10   INDL      C      D     STD   
2         2484  1984-01-31    1984     1   10   INDL      C      D     STD   
3         2484  1984-04-30    1984     2   10   INDL      C      D     STD   
4         2484  1984-07-31    1984     3   10   INDL      C      D     STD   
...        ...         ...     ...   ...  ...    ...    ...    ...     ...   
1075435  30165  1999-06-30    1999     2   12   INDL      C      D     STD   
1075436  30165  1999-09-30    1999     3   12   INDL      C      D     STD   
1075437  30165  1999-12-31    1999     4   12   INDL      C      D     STD   
1075438  30165  2000-03-31    2000     1   12   INDL      C      D     STD   
1075439  30165  2000-06-30    2000     2   12   INDL      C      D     STD   

           tic  ...  median_distance quarterly_median annual_me

#### After we removed duplicated rows we ended up with 2139 fyr_change_dummy ==1. However, after propogation we can expect 
#### fyr change count up to 4*2139=8556, because each dummy might have up to 3 more consecutive values set to 1. But we have 
#### 8171 < 8556, which is reasonable, because first, we might miss some entries for some quarters, second, propagation might have 
#### affected the same quarters. 

#### We propagated fyr_change_dummy because when fyr change happens, alternative expeced date calculation method applies not to that quarter only but the following three quarters too. After which only we can use our regular expected date calculation method. While propogating we made sure there are no duplicate rows (taking first instances only) and we checked the number of missing either fyearq or/and, no such rows were found.  


In [15]:
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')
df['fyearq'] = pd.to_numeric(df['fyearq'], errors='coerce')
df['fqtr'] = pd.to_numeric(df['fqtr'], errors='coerce')

grouped = df.groupby('tic', group_keys=False)

# Function to calculate consecutive differences
def calculate_diff(group):
    # Sort the data within each group by the fiscal year and quarter in ascending order
    group = group.sort_values(['fyearq', 'fqtr'])
    
    # Create a shifted column for the previous rdq
    group['prev_rdq'] = group['rdq'].shift(1)
    group['prev_fqtr'] = group['fqtr'].shift(1)
    group['prev_fyearq'] = group['fyearq'].shift(1)
    
    # Calculate the differences
    group['diff'] = group['rdq'] - group['prev_rdq']
    
    # Remove differences where quarters and years are not consecutive
    mask = (group['fqtr'] == 1) & (group['prev_fqtr'] == 4) & (group['fyearq'] == group['prev_fyearq'] + 1)
    mask |= (group['fqtr'] > 1) & (group['fqtr'] == group['prev_fqtr'] + 1) & (group['fyearq'] == group['prev_fyearq'])
    
    group['diff'] = group['diff'].where(mask)
    
    # Drop helper columns
    group = group.drop(columns=['prev_rdq', 'prev_fqtr', 'prev_fyearq'])
    
    return group

# Apply the function to each ticker group directly in the original DataFrame
df = grouped.apply(calculate_diff)
df.loc[df['fyr_change_dummy'] == 1, 'diff'] = pd.NaT
df = df.reset_index(drop=True)

#### Let's split the distance for quarterly and annual reports

In [16]:
df['quarterly_report'] = (df['fqtr'].isin([1, 2, 3])).astype(int)
df['annual_report'] = (df['fqtr'] == 4).astype(int)


In [17]:
df['quarter_distance'] = df['diff'].where(df['quarterly_report'] == 1, pd.NaT)
df['annual_distance'] = df['diff'].where(df['annual_report'] == 1, pd.NaT)
print(df[['tic', 'rdq', 'fyr', 'fyearq', 'fqtr', 'fyr_change_dummy', 'annual_distance', 'quarter_distance', 'diff']].head())


     tic        rdq  fyr  fyearq  fqtr  fyr_change_dummy annual_distance  \
0  0015B        NaT   10    1983     3                 0             NaT   
1  0015B 1984-01-16   10    1983     4                 0             NaT   
2  0015B 1984-03-15   10    1984     1                 0             NaT   
3  0015B 1984-05-25   10    1984     2                 0             NaT   
4  0015B 1984-08-27   10    1984     3                 0             NaT   

  quarter_distance    diff  
0              NaT     NaT  
1              NaT     NaT  
2          59 days 59 days  
3          71 days 71 days  
4          94 days 94 days  


In [18]:
df['market_cap'] = df['prccq'] / df['ajexq'] * df['cshoq']
df['lower_bound'] = 0.8 * df['market_cap']
df['upper_bound'] = 1.2 * df['market_cap']

In [19]:

# Initialize the 'median_distance' column with NaN values
df['median_distance'] = np.nan

# Ensure the 'median_distance' column is updated for fiscal year change rows
df['median_distance'] = np.where(df['fyr_change_dummy'] == 0, np.nan, df['median_distance'])

# Initialize variables to count instances with all empty and non-empty distances
num_instances_all_empty = 0
num_instances_non_empty = 0

# Initialize sets to store tickers with non-empty, empty, and mixed distances
non_empty_distances_tickers = set()
empty_distances_tickers = set()
mixed_distances_tickers = set()

# Filter rows where fiscal year change occurred
fyr_change_rows = df[df['fyr_change_dummy'] == 1]

# Iterate over rows where fiscal year change occurred
for index, fyr_change_row in fyr_change_rows.iterrows():
    # Determine if it's a quarterly or annual report
    is_quarterly_report = fyr_change_row['quarterly_report'] == 1

    # Filter similar market cap tickers excluding own ticker
    similar_cap_tickers = df[(df['market_cap'] >= fyr_change_row['lower_bound']) &
                             (df['market_cap'] <= fyr_change_row['upper_bound']) &
                             (df['tic'] != fyr_change_row['tic'])]

    # Determine relevant distance column and filter criteria
    distance_col = 'quarter_distance' if is_quarterly_report else 'annual_distance'
    year_col = 'fyearq'
    years = [fyr_change_row['fyearq'], fyr_change_row['fyearq'] + 1, fyr_change_row['fyearq'] - 1]

    # Extract distances
    distances = similar_cap_tickers[similar_cap_tickers[year_col].isin(years)][distance_col].dropna().apply(lambda x: pd.Timedelta(x).days)

    # Check if distances list is empty
    if distances.empty:
        num_instances_all_empty += 1
        if fyr_change_row['tic'] in non_empty_distances_tickers:
            mixed_distances_tickers.add(fyr_change_row['tic'])
        else:
            empty_distances_tickers.add(fyr_change_row['tic'])
    else:
        median_distance = distances.median()
        df.loc[index, 'median_distance'] = median_distance
        num_instances_non_empty += 1
        if fyr_change_row['tic'] in empty_distances_tickers:
            mixed_distances_tickers.add(fyr_change_row['tic'])
        else:
            non_empty_distances_tickers.add(fyr_change_row['tic'])

# Calculate counts for unique tickers
num_non_empty_distances_tickers = len(non_empty_distances_tickers - mixed_distances_tickers)
num_empty_distances_tickers = len(empty_distances_tickers - mixed_distances_tickers)
num_mixed_distances_tickers = len(mixed_distances_tickers)
num_unique_fyr_change_tickers = fyr_change_rows['tic'].nunique()

# Display the results
print(f"Number of instances with all empty distances: {num_instances_all_empty}")
print(f"Number of instances with non-empty distances: {num_instances_non_empty}")
print(f"Number of unique tickers with non-empty distances: {num_non_empty_distances_tickers}")
print(f"Number of unique tickers with all empty distances: {num_empty_distances_tickers}")
print(f"Number of unique tickers with mixed distances: {num_mixed_distances_tickers}")
print(f"Total number of unique tickers with fiscal year changes: {num_unique_fyr_change_tickers}")


Number of instances with all empty distances: 1146
Number of instances with non-empty distances: 7025
Number of unique tickers with non-empty distances: 1499
Number of unique tickers with all empty distances: 54
Number of unique tickers with mixed distances: 417
Total number of unique tickers with fiscal year changes: 1970


In [20]:
# Filter rows where fiscal year change occurred
fyr_change_rows = df[df['fyr_change_dummy'] == 1]

# Iterate over rows where fiscal year change occurred
for index, fyr_change_row in fyr_change_rows.iterrows():
    # Check if fiscal year change happened in quarterly or annual report
    is_quarterly_report = fyr_change_row['quarterly_report'] == 1

    # Determine which column to assign median distance
    median_distance_column = 'annual_median' if not is_quarterly_report else 'quarterly_median'

    # Update the median_distance column accordingly
    df.loc[index, median_distance_column] = fyr_change_row['median_distance']

# Set NaN for rows where fyr_change_dummy is 0
df.loc[df['fyr_change_dummy'] == 0, ['annual_median', 'quarterly_median']] = np.nan


In [21]:
summary_stats = df['median_distance'].describe()
formatted_stats = summary_stats.apply(lambda x: f'{x:.2f}')

print(formatted_stats)

count    7025.00
mean       96.12
std        11.27
min        38.00
25%        90.00
50%        91.00
75%       102.00
max       156.00
Name: median_distance, dtype: object


In [22]:
summary_stats = df['median_distance'].describe()
formatted_stats = summary_stats.apply(lambda x: f'{x:.2f}')

print(formatted_stats)

count    7025.00
mean       96.12
std        11.27
min        38.00
25%        90.00
50%        91.00
75%       102.00
max       156.00
Name: median_distance, dtype: object


In [23]:
annual_median_summary = df['annual_median'].describe()
print("Summary statistics for Annual Median:")
print(annual_median_summary)

quarterly_median_summary = df['quarterly_median'].describe()
print("\nSummary statistixcs for Quarterly Median:")
print(quarterly_median_summary)


Summary statistics for Annual Median:
count    1899.000000
mean      112.913902
std         8.588462
min        98.000000
25%       106.000000
50%       112.000000
75%       119.000000
max       156.000000
Name: annual_median, dtype: float64

Summary statistixcs for Quarterly Median:
count    5126.000000
mean       89.891728
std         1.859526
min        38.000000
25%        90.000000
50%        90.000000
75%        91.000000
max        96.000000
Name: quarterly_median, dtype: float64


In [24]:
summary_stats = df['median_distance'].describe()
formatted_stats = summary_stats.apply(lambda x: f'{x:.2f}')

print(formatted_stats)

count    7025.00
mean       96.12
std        11.27
min        38.00
25%        90.00
50%        91.00
75%       102.00
max       156.00
Name: median_distance, dtype: object


In [25]:
df['market_cap'] = df['prccq'] / df['ajexq'] * df['cshoq']


In [26]:
# Calculate upper and lower bounds
df['lower_bound'] = 0.8 * df['market_cap']
df['upper_bound'] = 1.2 * df['market_cap']

In [27]:
df['quarterly_report'] = (df['fqtr'].isin([1, 2, 3])).astype(int)
df['annual_report'] = (df['fqtr'] == 4).astype(int)


In [28]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & (df['market_cap'].isna())]
num_entries = len(filtered_df)

print(f'Number of entries where fyr_change_dummy == 1 and market_cap is empty: {num_entries}')

Number of entries where fyr_change_dummy == 1 and market_cap is empty: 1139


#### As we see instances which had fyr change, 1139 cases had empty market_caps. It means 1139 out of 1146 median distances that 
#### are uncalculated was due to **missing market_cap**.

#### We will find cshoq to be proportional value between its last and next reported values, otherwise it is reasonable that we can't calculate median distances. 

In [29]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & (df['cshoq'].isna())]
num_entries = len(filtered_df)

print(f'Number of entries where fyr_change_dummy == 1 and cshoq is empty: {num_entries}')

Number of entries where fyr_change_dummy == 1 and cshoq is empty: 1071


In [30]:
filtered_rows = df[(df['fyr_change_dummy'] == 1) & df['prccq'].isna()]
print(f"Number of rows with fiscal year change and 'prccq' missing: {len(filtered_rows)}")


Number of rows with fiscal year change and 'prccq' missing: 97


In [61]:
fr = df[(df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['prccq'].isna() ]
print(f"Number of rows with fiscal year change and both 'cshoq', 'prccq' missing: {len(fr)}")

Number of rows with fiscal year change and both 'cshoq', 'prccq' missing: 0


#### 1139 of missing market_cap, out of which 1071 is due to cshoq absence, 97 is from price 'prccq' absense. 

#### For now we will relieve this 97 missing price cases

#### Now we will look for one above and below non-empty 'cshoq'. Determining which year and quarter found cshoqs belong to, we calculate the proportionate cshoq where it's missing
#### Example: Missing cshoq is 2024 Q1, above non-empty cshoq 2023 Q4 = 7, below 2024 Q2 =8, then we fill in with 7.5
#### Missing cshoq is 2024 Q2, above non-empty cshoq 2024 Q2 = 1, below 2024 Q4 =2, then we fill in with 1.33. 
#### If above and below non-empty cshoqs are the same, then our cshoq is equal to that value. 

In [32]:
# Initialize above_cshoq, below_cshoq, above_datafqtr, and below_datafqtr as NaN
df['above_cshoq'] = np.nan
df['below_cshoq'] = np.nan
df['above_datafqtr'] = np.nan
df['below_datafqtr'] = np.nan

# Find rows where fyr_change_dummy is 1 and cshoq is NaN
missing_cshoq_indices = df[(df['fyr_change_dummy'] == 1) & df['cshoq'].isna()].index

# Iterate over the missing cshoq indices to find above_cshoq, below_cshoq, above_datafqtr, and below_datafqtr
for index in missing_cshoq_indices:
    tic = df.at[index, 'tic']
    current_datafqtr = df.at[index, 'datafqtr']
    
    # Find the row above with a non-missing cshoq value
    above_rows = df[(df['tic'] == tic) & (df['datafqtr'] < current_datafqtr)].sort_values(by='datafqtr', ascending=False)
    for _, above_row in above_rows.iterrows():
        if not pd.isna(above_row['cshoq']):
            df.at[index, 'above_cshoq'] = above_row['cshoq']
            df.at[index, 'above_datafqtr'] = above_row['datafqtr']
            break
    
    # Find the row below with a non-missing cshoq value
    below_rows = df[(df['tic'] == tic) & (df['datafqtr'] > current_datafqtr)].sort_values(by='datafqtr', ascending=True)
    for _, below_row in below_rows.iterrows():
        if not pd.isna(below_row['cshoq']):
            df.at[index, 'below_cshoq'] = below_row['cshoq']
            df.at[index, 'below_datafqtr'] = below_row['datafqtr']
            break

# Print rows where fyr_change_dummy is 1 and cshoq is NaN
print(df[(df['fyr_change_dummy'] == 1) & df['cshoq'].isna()])

C:\Users\balmeru\AppData\Local\Temp\ipykernel_7432\4092613696.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1988Q4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'above_datafqtr'] = above_row['datafqtr']
C:\Users\balmeru\AppData\Local\Temp\ipykernel_7432\4092613696.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1989Q4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'below_datafqtr'] = below_row['datafqtr']


         GVKEY    datadate  fyearq  fqtr  fyr indfmt consol popsrc datafmt  \
22        2484  1988-09-30    1989     1    6   INDL      C      D     STD   
23        2484  1988-12-31    1989     2    6   INDL      C      D     STD   
24        2484  1989-03-31    1989     3    6   INDL      C      D     STD   
1035     23929  1996-12-31    1997     1    9   INDL      C      D     STD   
2910      6726  1979-12-31    1980     1    9   INDL      C      D     STD   
...        ...         ...     ...   ...  ...    ...    ...    ...     ...   
1069738  11656  1991-05-31    1991     1    2   INDL      C      D     STD   
1069739  11656  1991-08-31    1991     2    2   INDL      C      D     STD   
1073140  11669  1994-01-31    1994     2    7   INDL      C      D     STD   
1073141  11669  1994-04-30    1994     3    7   INDL      C      D     STD   
1074392  11310  1991-06-30    1991     4    6   INDL      C      D     STD   

           tic  ...  market_cap lower_bound upper_bound median_

In [33]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & 
                 df['cshoq'].isna() & 
                 df['above_cshoq'].notna() & 
                 df['below_cshoq'].notna()]
count_both_na = filtered_df.shape[0]
print("Number of rows where both above_cshoq and below_cshoq are not  NaN:", count_both_na)


Number of rows where both above_cshoq and below_cshoq are not  NaN: 1017


In [34]:
df['diff_fqtr'] = np.nan

# Calculate diff_fqtr for rows where both below_datafqtr and above_datafqtr are given
mask = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['below_datafqtr'].notna() & df['above_datafqtr'].notna()

for index, row in df[mask].iterrows():
    # Extract year and quarter from below_datafqtr and above_datafqtr
    below_year, below_quarter = row['below_datafqtr'].split('Q')
    above_year, above_quarter = row['above_datafqtr'].split('Q')

    # Convert year and quarter to integers
    below_year = int(below_year)
    below_quarter = int(below_quarter)
    above_year = int(above_year)
    above_quarter = int(above_quarter)

    # Calculate the difference in quarters
    diff_fqtr = (below_year - above_year) * 4 + (below_quarter - above_quarter)

    # Update the diff_fqtr column with the calculated difference
    df.at[index, 'diff_fqtr'] = diff_fqtr


In [35]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & 
                 df['cshoq'].isna() & 
                 df['diff_fqtr'].isna() ]
count_both_na = filtered_df.shape[0]
print("Number of rows where diff_fqtr are  NaN:", count_both_na)



Number of rows where diff_fqtr are  NaN: 54


In [36]:
df['diff_cshoq'] = df.apply(lambda row: row['below_cshoq'] - row['above_cshoq'] if pd.notna(row['above_cshoq']) and pd.notna(row['below_cshoq']) else np.nan, axis=1)



In [37]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & 
                 df['cshoq'].isna() & 
                 df['diff_cshoq'].isna() ]
count_both_na = filtered_df.shape[0]
print("Number of rows where diff_cshoq are  NaN:", count_both_na)



Number of rows where diff_cshoq are  NaN: 54


In [38]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & 
                 df['cshoq'].isna() & 
                 df['above_datafqtr'].isna() &df['below_datafqtr'].isna()   ]
count_both_na = filtered_df.shape[0]
print("Number of rows where noth above_datafqtr and below_datafqtr are  NaN:", count_both_na)



Number of rows where noth above_datafqtr and below_datafqtr are  NaN: 3


In [39]:
df_original = pd.read_csv('/Users/balmeru/Downloads/QQQQ.csv')

# Slice the DataFrame to extract rows 76626 to 76640
df_subset = df_original.iloc[76626:76641][['tic', 'fyr', 'fqtr', 'cshoq', 'prccq', 'ajexq']]

print(df_subset)


          tic  fyr  fqtr   cshoq      prccq     ajexq
76626  CHLN.1   12     1  12.076   7.860000  1.000000
76627  CHLN.1   12     2  12.076   7.700000  1.000000
76628  CHLN.1   12     3  12.078   7.841000  1.000000
76629  CHLN.1   12     4  12.076   8.800000  1.000000
76630  CHLN.1   12     1  12.083   9.000000  1.000000
76631  CHLN.1   12     2  12.096  10.880000  1.000000
76632  CHLN.1   12     3  13.478  10.500000  1.000000
76633   CHAM.    6     4     NaN   6.250000  1.049999
76634   CHAM.    3     1     NaN   8.999999  1.049999
76635   CHAM.    3     2     NaN   9.249999  1.049999
76636   CHAM.    3     3     NaN  10.624999  1.049999
76637   CJHBQ    2     2  34.907   3.750000  0.800000
76638   CJHBQ    2     3  34.903   3.500000  0.800000
76639   CJHBQ    2     4  35.003   5.875000  0.800000
76640   CJHBQ    2     1  35.003   4.125000  0.800000


In [40]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & 
                 df['cshoq'].isna() & 
                 df['diff_cshoq'].notna() ]
count_both_na = filtered_df.shape[0]
print("Number of rows where diff_cshoq are not  NaN:", count_both_na)



Number of rows where diff_cshoq are not  NaN: 1017


In [41]:
df['new_cshoq'] = np.nan

# Calculate new_cshoq for rows where fyr_change_dummy is 1, cshoq is NaN, and diff_cshoq and diff_fqtr are not NaN
mask = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['diff_cshoq'].notna() & df['diff_fqtr'].notna()

for index, row in df[mask].iterrows():
    # Calculate the difference between datafqtr and above_datafqtr
    above_year, above_quarter = row['above_datafqtr'].split('Q')
    data_year, data_quarter = row['datafqtr'].split('Q')
    
    above_year = int(above_year)
    above_quarter = int(above_quarter)
    data_year = int(data_year)
    data_quarter = int(data_quarter)
    
    difference = (data_year - above_year) * 4 + (data_quarter - above_quarter)
    
    # Calculate new_cshoq value
    new_cshoq = row['above_cshoq'] + (row['diff_cshoq'] / row['diff_fqtr']) * difference
    
    # Update the new_cshoq column with the calculated value
    df.at[index, 'new_cshoq'] = new_cshoq

print(df[['tic', 'fyr', 'cshoq', 'above_cshoq', 'new_cshoq']][mask])


           tic  fyr  cshoq  above_cshoq  new_cshoq
22       0015B    6    NaN       11.762  11.773500
23       0015B    6    NaN       11.762  11.785000
24       0015B    6    NaN       11.762  11.796500
1035     0147A    9    NaN        6.548   6.551500
2910     2788A    9    NaN        8.436   8.447000
...        ...  ...    ...          ...        ...
1069738   YORK    2    NaN        8.477   8.700667
1069739   YORK    2    NaN        8.477   8.924333
1073140    ZLC    7    NaN       34.965  34.965000
1073141    ZLC    7    NaN       34.965  34.965000
1074392   ZRBA    6    NaN        1.454   1.454000

[1017 rows x 5 columns]


In [42]:
# Update new_cshoq for rows where fyr_change_dummy is 1, cshoq is NaN, and above_cshoq or below_cshoq is NaN
mask_above_nan = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['above_cshoq'].isna()
mask_below_nan = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna() & df['below_cshoq'].isna()

# Set new_cshoq equal to below_cshoq when above_cshoq is NaN
df.loc[mask_above_nan, 'new_cshoq'] = df.loc[mask_above_nan, 'below_cshoq']

# Set new_cshoq equal to above_cshoq when below_cshoq is NaN
df.loc[mask_below_nan, 'new_cshoq'] = df.loc[mask_below_nan, 'above_cshoq']

print(df[['tic', 'fyr', 'cshoq', 'above_cshoq', 'below_cshoq', 'new_cshoq']][mask_above_nan | mask_below_nan])


            tic  fyr  cshoq  above_cshoq  below_cshoq  new_cshoq
4682      3ACDX   12    NaN        2.100          NaN      2.100
9713      3BPCO   12    NaN       21.163          NaN     21.163
9714      3BPCO   12    NaN       21.163          NaN     21.163
37211    3STPHF    9    NaN        3.969          NaN      3.969
51136     5841A    9    NaN       85.800          NaN     85.800
51359     5853B    6    NaN        2.669          NaN      2.669
51360     5853B    6    NaN        2.669          NaN      2.669
52393     6033B    7    NaN        7.194          NaN      7.194
53626     6103B   12    NaN        3.416          NaN      3.416
55279     6733B    1    NaN        8.987          NaN      8.987
57425     7556A    6    NaN       16.111          NaN     16.111
57426     7556A    6    NaN       16.111          NaN     16.111
58521     8550B   12    NaN       56.286          NaN     56.286
58522     8550B   12    NaN       56.286          NaN     56.286
60280     9901B    8    N

### IMPORTANT: I filled in with above or below cshoq wherever either of them was missing!

In [43]:
# Replace original cshoq values with new_cshoq for rows where fyr_change_dummy is 1 and cshoq is NaN
mask_replace_cshoq = (df['fyr_change_dummy'] == 1) & df['cshoq'].isna()

# Update cshoq with new_cshoq values
df.loc[mask_replace_cshoq, 'cshoq'] = df.loc[mask_replace_cshoq, 'new_cshoq']

print(df[['tic', 'fyr', 'cshoq']][mask_replace_cshoq])


           tic  fyr      cshoq
22       0015B    6  11.773500
23       0015B    6  11.785000
24       0015B    6  11.796500
1035     0147A    9   6.551500
2910     2788A    9   8.447000
...        ...  ...        ...
1069738   YORK    2   8.700667
1069739   YORK    2   8.924333
1073140    ZLC    7  34.965000
1073141    ZLC    7  34.965000
1074392   ZRBA    6   1.454000

[1071 rows x 3 columns]


In [44]:
filtered_df = df[(df['fyr_change_dummy'] == 1) & 
                 df['cshoq'].isna()  ]
count_both_na = filtered_df.shape[0]
print("Number of rows where cshoq are  NaN:", count_both_na)



Number of rows where diff_fqtr are  NaN: 3


In [45]:
df['market_cap'] = df['prccq'] / df['ajexq'] * df['cshoq']

In [46]:
# Calculate upper and lower bounds
df['lower_bound'] = 0.8 * df['market_cap']
df['upper_bound'] = 1.2 * df['market_cap']

In [47]:
df['quarterly_report'] = (df['fqtr'].isin([1, 2, 3])).astype(int)
df['annual_report'] = (df['fqtr'] == 4).astype(int)


In [48]:
df['median_distance'] = np.nan

# Ensure the 'median_distance' column is updated for fiscal year change rows
df['median_distance'] = np.where(df['fyr_change_dummy'] == 0, np.nan, df['median_distance'])

# Initialize variables to count instances with all empty and non-empty distances
num_instances_all_empty = 0
num_instances_non_empty = 0

# Initialize sets to store tickers with non-empty, empty, and mixed distances
non_empty_distances_tickers = set()
empty_distances_tickers = set()
mixed_distances_tickers = set()

# Filter rows where fiscal year change occurred
fyr_change_rows = df[df['fyr_change_dummy'] == 1]

# Iterate over rows where fiscal year change occurred
for index, fyr_change_row in fyr_change_rows.iterrows():
    # Determine if it's a quarterly or annual report
    is_quarterly_report = fyr_change_row['quarterly_report'] == 1

    # Filter similar market cap tickers excluding own ticker
    similar_cap_tickers = df[(df['market_cap'] >= fyr_change_row['lower_bound']) &
                             (df['market_cap'] <= fyr_change_row['upper_bound']) &
                             (df['tic'] != fyr_change_row['tic'])]

    # Determine relevant distance column and filter criteria
    distance_col = 'quarter_distance' if is_quarterly_report else 'annual_distance'
    year_col = 'fyearq'
    years = [fyr_change_row['fyearq'], fyr_change_row['fyearq'] + 1, fyr_change_row['fyearq'] - 1]

    # Extract distances
    distances = similar_cap_tickers[similar_cap_tickers[year_col].isin(years)][distance_col].dropna().apply(lambda x: pd.Timedelta(x).days)

    # Check if distances list is empty
    if distances.empty:
        num_instances_all_empty += 1
        if fyr_change_row['tic'] in non_empty_distances_tickers:
            mixed_distances_tickers.add(fyr_change_row['tic'])
        else:
            empty_distances_tickers.add(fyr_change_row['tic'])
    else:
        median_distance = distances.median()
        df.loc[index, 'median_distance'] = median_distance
        num_instances_non_empty += 1
        if fyr_change_row['tic'] in empty_distances_tickers:
            mixed_distances_tickers.add(fyr_change_row['tic'])
        else:
            non_empty_distances_tickers.add(fyr_change_row['tic'])

# Calculate counts for unique tickers
num_non_empty_distances_tickers = len(non_empty_distances_tickers - mixed_distances_tickers)
num_empty_distances_tickers = len(empty_distances_tickers - mixed_distances_tickers)
num_mixed_distances_tickers = len(mixed_distances_tickers)
num_unique_fyr_change_tickers = fyr_change_rows['tic'].nunique()

# Display the results
print(f"Number of instances with all empty distances: {num_instances_all_empty}")
print(f"Number of instances with non-empty distances: {num_instances_non_empty}")
print(f"Number of unique tickers with non-empty distances: {num_non_empty_distances_tickers}")
print(f"Number of unique tickers with all empty distances: {num_empty_distances_tickers}")
print(f"Number of unique tickers with mixed distances: {num_mixed_distances_tickers}")
print(f"Total number of unique tickers with fiscal year changes: {num_unique_fyr_change_tickers}")


Number of instances with all empty distances: 108
Number of instances with non-empty distances: 8063
Number of unique tickers with non-empty distances: 1932
Number of unique tickers with all empty distances: 9
Number of unique tickers with mixed distances: 29
Total number of unique tickers with fiscal year changes: 1970


In [49]:
summary_stats = df['median_distance'].describe()
formatted_stats = summary_stats.apply(lambda x: f'{x:.2f}')

print(formatted_stats)

count    8063.00
mean       95.46
std        10.85
min        38.00
25%        90.00
50%        90.00
75%        91.00
max       156.00
Name: median_distance, dtype: object


In [50]:
# Filter rows where fiscal year change occurred
fyr_change_rows = df[df['fyr_change_dummy'] == 1]

# Iterate over rows where fiscal year change occurred
for index, fyr_change_row in fyr_change_rows.iterrows():
    # Check if fiscal year change happened in quarterly or annual report
    is_quarterly_report = fyr_change_row['quarterly_report'] == 1

    # Determine which column to assign median distance
    median_distance_column = 'annual_median' if not is_quarterly_report else 'quarterly_median'

    # Update the median_distance column accordingly
    df.loc[index, median_distance_column] = fyr_change_row['median_distance']

# Set NaN for rows where fyr_change_dummy is 0
df.loc[df['fyr_change_dummy'] == 0, ['annual_median', 'quarterly_median']] = np.nan


In [51]:
annual_median_summary = df['annual_median'].describe()
print("Summary statistics for Annual Median:")
print(annual_median_summary)

quarterly_median_summary = df['quarterly_median'].describe()
print("\nSummary statistixcs for Quarterly Median:")
print(quarterly_median_summary)


Summary statistics for Annual Median:
count    1954.000000
mean      112.930911
std         8.585022
min        98.000000
25%       106.000000
50%       112.000000
75%       119.000000
max       156.000000
Name: annual_median, dtype: float64

Summary statistixcs for Quarterly Median:
count    6109.000000
mean       89.872974
std         1.756219
min        38.000000
25%        89.000000
50%        90.000000
75%        91.000000
max        96.000000
Name: quarterly_median, dtype: float64


#### Overall we have 8171 cases with fyr change. We calculated median distances for 8063 cases. 
#### 1139 out of 1146 median distances was due to missing market cap. 
#### 97 cases didn't have prccq when fyr_change_dummy == 1, thus didn't have market cap to be calculated. 


In [52]:
# Read the CSV file
filtered_df = df[(df['fyr_change_dummy'] == 1) & df['prccq'].isna()]

# Select the specific columns
filtered_columns_df = filtered_df[['tic', 'fyr', 'fqtr', 'cshoq', 'prccq', 'ajexq']]

# Print the filtered DataFrame
print(len(filtered_columns_df))



97


In [58]:
df.to_csv('/Users/balmeru/Downloads/with_median_distances.csv', index=False)

print("Selected rows have been saved to 'with_median_distances.csv'.")

Selected rows have been saved to 'with_median_distances.csv'.


In [59]:
num_rows_shape = df.shape[0]
print(f"Number of rows in the DataFrame (using shape): {num_rows_shape}")

Number of rows in the DataFrame (using shape): 1075440
